<a href="https://colab.research.google.com/github/vatsaaa/mtech/blob/main/semester_1/03_assignments/mfml/Assignment_01_Q01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Python function to create a random matrix, but without using libraries

In [ ]:
def generate_random_matrix(rows: int, columns: int, decimal_allowed: bool=False):
    matrix = []

    for _ in range(rows):
      if decimal_allowed:
        row = [random.choice([random.randint(1, 100), round(random.uniform(0, 1))]) for _ in range(columns)]
      else:

        row = [random.randint(1, 100) for _ in range(columns)]

        matrix.append(row)

    return matrix

  generate_random